In [ ]:
import pandas as pd
import numpy as np
import pyomo.environ as pe
import pyomo.opt as po
import time

np.float_ = np.float64
np.complex_ = np.complex128

In [ ]:
import os
import subprocess

# Check the operating system
os_name = os.name
print(f"Operating System: {os_name}")

# Check if GLPK is installed
def check_glpk_installed():
    try:
        result = subprocess.run(['glpsol', '--version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            print("GLPK is installed.")
        else:
            print("GLPK is not installed.")
            install_glpk()
    except FileNotFoundError:
        print("GLPK is not installed.")
        install_glpk()

def install_glpk():
    try:
        if os_name == 'posix':
            distro = subprocess.run(['lsb_release', '-is'], stdout=subprocess.PIPE, text=True).stdout.strip().lower()
            if 'ubuntu' in distro or 'debian' in distro:
                subprocess.run(['sudo', 'apt-get', 'install', 'glpk-utils', '-y'], check=True)
                print("GLPK installation completed on Ubuntu/Debian.")
            elif 'fedora' in distro or 'centos' in distro:
                subprocess.run(['sudo', 'dnf', 'install', 'glpk', '-y'], check=True)
                print("GLPK installation completed on Fedora/CentOS.")
            elif 'darwin' in distro:
                subprocess.run(['brew', 'install', 'glpk'], check=True)
                print("GLPK installation completed on macOS.")
            else:
                print("Unsupported Linux distribution. Please install GLPK manually.")
        elif os_name == 'nt':
            print("Please install GLPK manually from: https://winglpk.sourceforge.net/")
        else:
            print("Unsupported operating system.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to install GLPK: {e}")

check_glpk_installed()


In [ ]:
# Cargar datos
price = pd.read_csv('../data/raw/PriceCurve_SE4_2021.csv', sep=';')
co2_pro = pd.read_csv('../data/raw/production_emissions.csv')
co2_con = pd.read_csv('../data/raw/consumption_emissions.csv')
pv = pd.read_csv('../data/raw/oskarshamnpvprod.csv')
load = pd.read_csv('../data/raw/LoadCurveo.csv', sep=',')

data = load
data['Price'] = price['Grid_Price']
data['CO_2_eq'] = co2_pro['carbon_intensity_production_avg']
data['solar_PV'] = pv

# Convertir unidades de MW a kW
data['Load'] = data['Load'] * 1000
# Los datos solares ya están en kW, pero necesitan ser multiplicados por el factor de escala
data['solar_PV'] = data['solar_PV'] # este factor solar luego se intera en el procesado de datos dentro del 'for'
data['Price'] = data['Price'] / 1000
data['CO_2_eq'] = data['CO_2_eq'] / 1000

data['Hour'] = data['Hour'].astype('int')

# Duplicar filas
duplicated_rows = pd.concat([data.iloc[[-1]]] * 49, ignore_index=True)
data = pd.concat([data, duplicated_rows], ignore_index=True)

In [ ]:

# Parámetros de entrada
# power_PV=       [0.01, 0.03, 0.05, 0.1, 0.2]
# power_BESS=     [1, 3, 5, 10, 20]
# time_BESS=      [0, 1, 2, 3, 4]
# optimise_for=   ['Price', 'CO_2_eq']
power_PV = [0.01, 0.03, 0.1]
power_BESS = [1, 3, 10]     # Aqui se podria agregar un multiplo de carga para otra bateria
time_BESS = [0, 2, 4]       # Aqui de igual forma pero para el tiempo de carga
optimise_for = ['price', 'co2']

# Crear lista de casos de referencia
referenceCases = [
    [pv, bess, bess * time, curve]
    for pv in power_PV
    for bess in power_BESS
    for time in time_BESS
    for curve in optimise_for
]

# Parámetros del escenario
params = {
    'pv_price': 80,  # https://data.nrel.gov/submissions/53 en EUR/kW
    'bess_price': 200,  # https://doi.org/10.1016/j.solener.2018.08.061 en EUR/kWh, ajustado por disminuciones de precio
    'pv_opex': 3,  # EUR/kWh -> referencia en excel
    'bess_opex': 6,  # EUR/kWh -> referencia en excel
    'pv_co2': 33,  # kgCO2eq/kW_powerDC -> referencia en excel
    'bess_co2': 100,  # kgCO2eq/kWh_capacity -> referencia en excel
    'pv_opex_co2': 0,  # kgCO2eq/kW_powerDC -> suposición
    'bess_opex_co2': 0,  # kgCO2eq/kW_powerDC -> suposición
    'discount_rate': 0.0485,  # suposición
    'lifetime_project': 32,  # para la vida útil del proyecto
    'lifetime_bess': 8,  # para la vida útil del BESS
    'degradation_rate': 0.025  # suposición (basado en alcanzar 80% SoH en 8 años)
}

# Definir conjuntos y parámetros globales
flows_global = [
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
]

hours_global = list(range(8760 + 24))
time_window = 48
num_iterations = len(hours_global) / time_window

# La disponibilidad de la red para cada hora se establece en 1000 MW, debería ser más que suficiente para la demanda de nuestro modelo y cualquier demanda potencial de BESS para el arbitraje de energía
grid_production = 1000000

# Rango de casos para el optimizador de despacho
pv_load_multiples = [0, 1, 2, 3]
bess_load_multiples = [1, 2, 3]
bess_duration = [0, 1, 2, 3]



# Parámetros adicionales BESS
efficiency_charge = 0.98
efficiency_discharge = 0.96
efficiency_inverter = 0.97

# qué múltiplo de la carga limitaremos la inyección de la red.
# El valor predeterminado es 20%
overreach_capacity = 20
demand_multiple = float(data['Load'].max()) * (1 + overreach_capacity / 100)


In [ ]:
# Crear el directorio si no existe
output_dir = '../data/interim/CaseB'
os.makedirs(output_dir, exist_ok=True)

# Función para verificar si un archivo existe
def file_exists(filename):
    try:
        with open(filename, 'r'):
            pass
    except FileNotFoundError:
        return False
    return True

In [ ]:


for bess_cap in bess_duration:
    for bess_multiple in bess_load_multiples:
        for solar_multiple in pv_load_multiples:
            for curve in optimise_for:
                if not file_exists(f'{output_dir}/{curve}_{int(solar_multiple)}_{bess_multiple}_{bess_cap}.csv'):
                    results = pd.DataFrame(index=hours_global, columns=flows_global + ['SoC'])

                    # En kW, múltiplo solar multiplicado por la carga máxima (== 3000 kW) y dividido por la producción base de PV (100MW == 100 000 kW)
                    p_solar = (solar_multiple * 3000) / 100000
                    # En kW, múltiplo de BESS multiplicado por la carga máxima (== 3000 kW)
                    p_bess = bess_multiple * 3000
                    # En horas
                    t_bess = bess_cap
                    if t_bess != 0:
                        p_bess = bess_multiple * 3000  # Convertir de MW a kW
                    else:
                        p_bess = 0
                    # En kWh
                    e_bess = p_bess * t_bess
                    soc_initial = e_bess * 0.5

                    start_time = time.time()
                    print(f'PV Multiple: {solar_multiple}x Peak Load [MW] || BESS Multiples: {bess_multiple}x Peak Load [MW]; {bess_cap} hr (at rated power) == {bess_multiple * 3 * bess_cap} [MWh]')
                    print(f'Optimised for: {curve}-curve')

                    for iteration in range(1, int(num_iterations) + 1):
                        hours = list(range((iteration - 1) * time_window, iteration * time_window))

                        demand = {hour: data['Load'][hour] for hour in hours}
                        pv_production = {hour: data['solar_PV'][hour] * p_solar for hour in hours}

                        costs_keys = [(flow, hour) for flow in flows_global for hour in hours]

                        costs_economic = {
                            key: data['Price'][key[1]] if key[0] in ['P_Grid_to_Load', 'P_Grid_to_BESS'] else
                            -data['Price'][key[1]] / 1000 if key[0] in ['P_PV_to_Grid', 'P_BESS_to_Grid'] else
                            0 if key[0] in ['P_PV_to_Load', 'P_PV_to_BESS', 'P_BESS_to_Load'] else
                            1000 / 1000 if key[0] == 'P_PV_curtailment' else
                            0
                            for key in costs_keys
                        }

                        costs_environmental = {
                            key: data['CO_2_eq'][key[1]] if key[0] in ['P_Grid_to_Load', 'P_Grid_to_BESS'] else
                            -data['CO_2_eq'][key[1]] / 1000 if key[0] in ['P_PV_to_Grid', 'P_BESS_to_Grid'] else
                            0 if key[0] in ['P_PV_to_Load', 'P_PV_to_BESS', 'P_BESS_to_Load'] else
                            1000 / 1000 if key[0] == 'P_PV_curtailment' else
                            0
                            for key in costs_keys
                        }

                        model = pe.ConcreteModel()
                        model.flows = pe.Set(initialize=flows_global, ordered=True)
                        model.hours = pe.Set(initialize=hours, ordered=True)

                        model.grid_production = pe.Param(initialize=grid_production)
                        model.demand = pe.Param(model.hours, initialize=demand)
                        model.pv_production = pe.Param(model.hours, initialize=pv_production)

                        model.Efficiency_charge = pe.Param(model.hours, initialize=efficiency_charge)
                        model.Efficiency_inverter = pe.Param(model.hours, initialize=efficiency_discharge)
                        model.Efficiency_discharge = pe.Param(model.hours, initialize=efficiency_inverter)

                        model.SoCmin = pe.Param(initialize=e_bess * 0.1)
                        model.SoCmax = pe.Param(initialize=e_bess * 0.9)
                        model.MaxCharge = pe.Param(initialize=p_bess)

                        if iteration == 1:
                            model.SoCinitial = pe.Param(initialize=soc_initial)
                        else:
                            last_SoC_value = float(results['SoC'].iloc[-1]) if not pd.isna(results['SoC'].iloc[-1]) else e_bess * 0.1
                            model.SoCinitial = pe.Param(initialize=last_SoC_value)

                        model.arbitrageLimit = pe.Param(model.hours, initialize=demand_multiple)
                        model.costs = pe.Param(model.flows, model.hours, initialize=costs_economic if curve == 'price' else costs_environmental, default=1)

                        model.p = pe.Var(model.flows, model.hours, domain=pe.NonNegativeReals)
                        model.SoC = pe.Var(model.hours, domain=pe.Reals, bounds=(model.SoCmin, model.SoCmax))
                        model.Sw_charge = pe.Var(model.hours, domain=pe.Binary)
                        model.Sw_discharge = pe.Var(model.hours, domain=pe.Binary)

                        minCosts = sum(model.p[f, t] * model.costs[f, t] for f in model.flows for t in model.hours)
                        model.objective = pe.Objective(sense=pe.minimize, expr=minCosts)

                        model.demandRule = pe.Constraint(model.hours, rule=lambda model, t: (
                            model.p['P_PV_to_Load', t] * efficiency_inverter +
                            model.p['P_BESS_to_Load', t] * (efficiency_discharge * efficiency_inverter) +
                            model.p['P_Grid_to_Load', t]
                        ) == model.demand[t])

                        model.pvProductionRule = pe.Constraint(model.hours, rule=lambda model, t: (
                            model.p['P_PV_to_Load', t] +
                            model.p['P_PV_to_BESS', t] +
                            model.p['P_PV_to_Grid', t] +
                            model.p['P_PV_curtailment', t]
                        ) == model.pv_production[t])

                        model.arbitrageRule = pe.Constraint(model.hours, rule=lambda model, t: (
                            model.p['P_BESS_to_Grid', t] * (efficiency_discharge * efficiency_inverter) +
                            model.p['P_PV_to_Grid', t] * efficiency_inverter
                        ) <= model.arbitrageLimit[t])

                        model.gridRule = pe.Constraint(model.hours, rule=lambda model, t: (
                            model.p['P_Grid_to_Load', t] +
                            model.p['P_Grid_to_BESS', t]
                        ) <= model.grid_production)

                        model.chargeRule = pe.Constraint(model.hours, rule=lambda model, t: (
                            model.p['P_Grid_to_BESS', t] +
                            model.p['P_PV_to_BESS', t]
                        ) <= model.Sw_charge[t] * model.MaxCharge)

                        model.dischargeRule = pe.Constraint(model.hours, rule=lambda model, t: (
                            model.p['P_BESS_to_Grid', t] +
                            model.p['P_BESS_to_Load', t]
                        ) <= model.Sw_discharge[t] * model.MaxCharge)

                        model.limitValidation = pe.Constraint(model.hours, rule=lambda model, t: (
                            model.Sw_charge[t] + model.Sw_discharge[t] <= 1))

                        model.charge_state = pe.Constraint(model.hours, rule=lambda model, t: (
                            model.SoC[t] == model.SoCinitial + (
                                (model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)
                            ) - (
                                (model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t]) / (efficiency_discharge * efficiency_inverter)
                            ) if t == model.hours.first() else
                            model.SoC[t] == model.SoC[t-1] + (
                                (model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)
                            ) - (
                                (model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t]) / (efficiency_discharge * efficiency_inverter)
                            )
                        ))

                        solver = po.SolverFactory('glpk')
                        solver.options['tmlim'] = 15
                        solver.options['mipgap'] = 0.05
                        modelresults = solver.solve(model, tee=False)

                        for flow in flows_global + ['SoC']:
                            for hour in hours:
                                results.at[hour, flow] = model.SoC[hour].value if flow == 'SoC' else model.p[flow, hour].value

                        results['sum_power_flows'] = results[['P_PV_to_Load', 'P_BESS_to_Load', 'P_Grid_to_Load']].sum(axis=1)

                        for param in model.component_data_objects(pe.Param, active=True):
                            model.del_component(param)
                        for variable in model.component_data_objects(pe.Var, active=True):
                            model.del_component(variable)
                        for cons in model.component_data_objects(pe.Constraint, active=True):
                            model.del_component(cons)
                        for obj in model.component_data_objects(pe.Objective, active=True):
                            model.del_component(obj)
                        model.del_component(model.hours)
                        model.del_component(model.flows)

                    results.reset_index(inplace=True)
                    results.rename(columns={'index': 'Hour'}, inplace=True)
                    results.to_csv(f'{output_dir}/{curve}_{int(solar_multiple)}_{bess_multiple}_{t_bess}.csv', sep=',', index=False)

                    results = []
                    end_time = time.time()
                    execution_time = end_time - start_time
                    hours, remainder = divmod(execution_time, 3600)
                    minutes, seconds = divmod(remainder, 60)
                    print(f"Execution time: {int(hours):02}:{int(minutes):02}:{int(seconds):02}")
                    print('-' * 50)